# From the previous notebook we saw that each of the mouse samples had different feature counts, now we can look further into the data to figure out how they were expressed differently based on those feature counts. We will use R and the packages Rsubread and DESeq2 for this analysis (use following command to load R that already has the packages installed: 
# module load R-4.1.2

In [1]:
MIAMIID = !echo $USER
MIAMIID = str(MIAMIID)
MIAMIID = MIAMIID[2:len(MIAMIID)-2]
print(MIAMIID)

schroe51


## We are going to read in counts.csv from last notebook in an R script to use DESeq2

In [2]:
%mkdir -p /home/{MIAMIID}/test/DESeq2
%cd /home/{MIAMIID}/test/DESeq2

/home/schroe51/test/DESeq2


## We need to now generate a metadata file for our run data in order for DESeq2 to perform analysis. Refer to the following table to see how we are constructing our metadata. This data was derived from the metadata of the runs for this experiment on the NCBI run selector.

| SRR ID | Treatment |
| --- | --- |
| SRR24062652 | KO |
| SRR24062653 | KO |
| SRR24062654 | KO |
| SRR24062655 | WT |
| SRR24062656 | WT |
| SRR24062657 | WT |

In [3]:
import csv

pathToNewFile = "/home/"+ MIAMIID + "/test/DESeq2/metadata.csv"

header = ['SRR.ID', 'Treatment']
data = ["SRR24062652", "KO", 
        "SRR24062653", "KO", 
        "SRR24062654", "KO", 
        "SRR24062655", "WT", 
        "SRR24062656", "WT", 
        "SRR24062657", "WT"]

# open the file in the write mode
with open(pathToNewFile, 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    for i in range(0, len(data) - 1, 2):
        writer.writerow(data[i: i+2])


In [4]:
!cat /home/{MIAMIID}/test/DESeq2/metadata.csv

## Now that we have our meta data, we can write our R script to use DESeq2 to get statistics about differential expression.

In [6]:
pathToCountData = "/home/"+ MIAMIID + "/test/featureCounts/counts_short.csv"
pathToMetaData = "/home/"+ MIAMIID + "/test/DESeq2/metadata.csv"

DESeq2Script = '''
suppressMessages(library( "DESeq2" ))
suppressMessages(library(ggplot2))
suppressMessages(library(ggpubr))
suppressMessages(library(ggrepel))

countData <- read.csv("''' + pathToCountData + '''", header = TRUE, sep = ",")
metaData <- read.csv("''' + pathToMetaData + '''", header = TRUE, sep = ",")

# construct DESeq data object
dds <- DESeqDataSetFromMatrix(countData=countData, 
                              colData=metaData, 
                              design=~Treatment, tidy = TRUE)

# take a look at the DESeq data object
dds

# 
dds <- DESeq(dds)

res <- results(dds)
head(results(dds, tidy=TRUE))

summary(res)

res <- res[order(res$padj),]
head(res)

# plotting counts for the lowest p-value genes

par(mfrow=c(2,3))

d1 <- plotCounts(dds, gene="ENSMUSG00000110469", intgroup="Treatment", returnData=TRUE)
d2 <- plotCounts(dds, gene="ENSMUSG00000044424", intgroup="Treatment", returnData=TRUE)
d3 <- plotCounts(dds, gene="ENSMUSG00000107383", intgroup="Treatment", returnData=TRUE)
d4 <- plotCounts(dds, gene="ENSMUSG00000114547", intgroup="Treatment", returnData=TRUE)
d5 <- plotCounts(dds, gene="ENSMUSG00000057657", intgroup="Treatment", returnData=TRUE)
d6 <- plotCounts(dds, gene="ENSMUSG00000084149", intgroup="Treatment", returnData=TRUE)

plot_pval_1 <- ggplot(d1, aes(x = Treatment, y = count, color = Treatment)) + 
  geom_point(position=position_jitter(w = 0.1,h = 0)) +
  theme_bw() +
  ggtitle("ENSMUSG00000110469") +
  theme(plot.title = element_text(hjust = 0.5))
  
plot_pval_2 <- ggplot(d2, aes(x = Treatment, y = count, color = Treatment)) + 
  geom_point(position=position_jitter(w = 0.1,h = 0)) +
  theme_bw() +
  ggtitle("ENSMUSG00000044424") +
  theme(plot.title = element_text(hjust = 0.5))
  
plot_pval_3 <- ggplot(d3, aes(x = Treatment, y = count, color = Treatment)) + 
  geom_point(position=position_jitter(w = 0.1,h = 0)) +
  theme_bw() +
  ggtitle("ENSMUSG00000107383") +
  theme(plot.title = element_text(hjust = 0.5))
  
plot_pval_4 <- ggplot(d4, aes(x = Treatment, y = count, color = Treatment)) + 
  geom_point(position=position_jitter(w = 0.1,h = 0)) +
  theme_bw() +
  ggtitle("ENSMUSG00000114547") +
  theme(plot.title = element_text(hjust = 0.5))
  
plot_pval_5 <- ggplot(d5, aes(x = Treatment, y = count, color = Treatment)) + 
  geom_point(position=position_jitter(w = 0.1,h = 0)) +
  theme_bw() +
  ggtitle("ENSMUSG00000057657") +
  theme(plot.title = element_text(hjust = 0.5))
  
plot_pval_6 <- ggplot(d6, aes(x = Treatment, y = count, color = Treatment)) + 
  geom_point(position=position_jitter(w = 0.1,h = 0)) +
  theme_bw() +
  ggtitle("ENSMUSG00000084149") +
  theme(plot.title = element_text(hjust = 0.5))

arrange <- ggarrange(plot_pval_1, plot_pval_2, plot_pval_3, plot_pval_4, plot_pval_5, plot_pval_6, ncol = 2, nrow = 3)
ggsave("6mostSigGenes.png", arrange)

# volcano plot

#reset par
par(mfrow=c(1,1))

# Make a basic volcano plot

# res <- as.data.frame(res)
# print(colnames(res))
# 
# res$diffexpressed <- "NO"
# res$diffexpressed[res$log2FoldChange > 0.6 & res$pvalue < 0.05] <- "UP"
# res$diffexpressed[res$log2FoldChange < -0.6 & res$pvalue < 0.05] <- "DOWN"
# 
# volcano <- ggplot(data=res, aes(x=log2FoldChange, y=-log10(pvalue), col=diffexpressed, label=res.index)) +
#           geom_point() + 
#           theme_minimal() +
#           geom_text_repel() +
#           scale_color_manual(values=c("blue", "black", "red")) +
#           geom_vline(xintercept=c(-0.6, 0.6), col="red") +
#           geom_hline(yintercept=-log10(0.05), col="red")

volcano <- with(res, plot(log2FoldChange, -log10(pvalue), pch=20, main="Volcano plot"))

# Add colored points: blue if padj<0.01, red if log2FC>1 and padj<0.05)
volcano <- with(subset(res, padj<.05 & log2FoldChange< -1), points(log2FoldChange, -log10(pvalue), pch=20, col="blue"))
volcano <- with(subset(res, padj<.05 & log2FoldChange>1), points(log2FoldChange, -log10(pvalue), pch=20, col="red"))

ggsave("volcano.png", volcano)

# PCA
vsdata <- vst(dds, blind=FALSE)
pcaData <- plotPCA(vsdata, intgroup="Treatment", returnData=TRUE)
pcaPlot <- ggplot(pcaData, aes(x=PC1,y=PC2,col=Treatment,label=name)) + geom_point() + geom_text_repel()
ggsave("pca.png", pcaPlot)
'''
with open('DESeq2_analysis.R', 'w') as file:
  file.write(DESeq2Script)

In [7]:
%%sh
Rscript DESeq2_analysis.R

class: DESeqDataSet 
dim: 55487 6 
metadata(1): version
assays(1): counts
rownames(55487): ENSMUSG00000102693 ENSMUSG00000064842 ...
  ENSMUSG00000064371 ENSMUSG00000064372
rowData names(0):
colnames(6): SRR24062652 SRR24062653 ... SRR24062656 SRR24062657
colData names(2): SRR.ID Treatment
                 row baseMean log2FoldChange    lfcSE      stat    pvalue padj
1 ENSMUSG00000102693  0.00000             NA       NA        NA        NA   NA
2 ENSMUSG00000064842  0.00000             NA       NA        NA        NA   NA
3 ENSMUSG00000051951  6.57908      0.6760391 1.202132 0.5623666 0.5738663   NA
4 ENSMUSG00000102851  0.00000             NA       NA        NA        NA   NA
5 ENSMUSG00000103377  0.00000             NA       NA        NA        NA   NA
6 ENSMUSG00000104017  0.00000             NA       NA        NA        NA   NA

out of 22173 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 188, 0.85%
LFC < 0 (down)     : 473, 2.1%
outliers [1]       : 146, 

Warning message:
In DESeqDataSet(se, design = design, ignoreRank) :
  some variables in design formula are characters, converting to factors
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image


# Now that we have finished the statistical model, let us move and look at some of the plots that were generated

In [8]:
%mkdir -p /home/{MIAMIID}/test/DESeq2/plots
!mv *.png /home/{MIAMIID}/test/DESeq2/plots
!mv *.pdf /home/{MIAMIID}/test/DESeq2/plots
%cd /home/{MIAMIID}/test/DESeq2/plots
!ls

/home/schroe51/test/DESeq2/plots
6mostSigGenes.png  pca.png  Rplots.pdf	volcano.png


# Congratulations! You are now done generating data, you may now interpret the results in the output and the graphs generated by the R script.